**import**

In [2]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack ..

**pyspark**

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

**copy wiki**

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

# AUTH_TIMEOUT = 30
AUTH_TIMEOUT = 30000000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [7]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'acoustic-cargo-414010'
!gcloud config set project {project_id}

data_bucket_name = 'noam209263805'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


Updated property [core/project].
Copying gs://noam209263805/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [8]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# parquetFile.show()

In [9]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd


###  **Text & Query Preprocces**



In [10]:
# stop words

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


In [11]:
# tokenize and stemming

def tokenize_stemming(text):
    """
    Remove stopword and stem the word

    :param text: str

    :return list of clean tokens after stemming
    """
    list_token = []
    english_stopwords = frozenset(stopwords.words('english'))
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    stemmer = PorterStemmer()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    for token in tokens:
        if token not in english_stopwords:
            list_token.append(stemmer.stem(token))
    return list_token


### **Inverted Index**

**term frequency**

In [12]:
# tf
# normalizion by max frequency term

def term_frequency(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
        text: str
            Text of one document
        id: int
            Document id
    Returns:
    --------
        List of tuples
            A list of (token, (doc_id, normalized_tf)) pairs
            for example: [("Anarchism", (12, 0.5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    # Count the frequency of each word that is not a stopword
    word_freq = {}
    max_freq = 0
    # print(tokens)
    for token in tokens:
        if token not in all_stopwords:
            if token not in word_freq:
                word_freq[token] = 1
            else:
                word_freq[token] += 1
            # max_freq = max(max_freq, word_freq[token])


    # Normalize the frequency of each word
    normalized_word_freq = {}

    for token, freq in word_freq.items():
        normalized_freq = freq
        normalized_word_freq[token] = normalized_freq

    lst_tuples = [(token, (id, freq)) for token, freq in normalized_word_freq.items()]

    return lst_tuples

In [13]:
# test for word count
text = "This is a test to check the words count function actually count words"
id = 0
print(term_frequency(text, id))
word_count_results = term_frequency(text, id)
assert word_count_results[0][0] == 'test'
assert len(word_count_results) == 6
assert word_count_results[2][1][1] == word_count_results[2][1][1]
assert type(word_count_results[0][0]) == str
assert type(word_count_results[0][1]) == tuple

[('test', (0, 1)), ('check', (0, 1)), ('words', (0, 2)), ('count', (0, 2)), ('function', (0, 1)), ('actually', (0, 1))]


In [17]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

In [18]:
word_counts = doc_text_pairs.flatMap(lambda x: term_frequency(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>10)


**df**


In [19]:
# df

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  rdd_token_doc_freq = postings.map(lambda x: (x[0], len(x[1]))).reduceByKey(lambda a, b: a + b)

  return rdd_token_doc_freq

Doc len calc


In [20]:
# doc len

def doc_lenght(text, doc_id):
  tokens = tokenize_stemming(text)
  return((id,len(tokens)))



## Partitioning and writing the index

In [21]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  def map_to_buckets(pair):
        word, posting_list = pair
        bucket_id = token2bucket_id(word)
        return bucket_id, [(word, posting_list)]

  def reduce_to_disk(bucket, word_posting_lists):
    # Write posting lists for the bucket to disk
    posting_locs = InvertedIndex.write_a_posting_list((bucket, word_posting_lists))
    return posting_locs

  # Partition the postings into buckets
  buckets = postings.map(map_to_buckets)

  # Group postings by bucket ID
  grouped_buckets = buckets.reduceByKey(lambda x, y: x + y)

  # Write each bucket to disk and get posting locations
  bucket_posting_locs = grouped_buckets.map(lambda x: reduce_to_disk(*x))

  return bucket_posting_locs



create an inverted index instance

In [22]:
posting_locs_list = partition_postings_and_write(postings_filtered).collect()

# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)


# df
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# number of docs
N = doc_text_pairs.count()

# save each doc its len
dl_rdd = doc_text_pairs.map(lambda x: (x[1], len(tokenize_stemming(x[0]))))
dl = dl_rdd.collectAsMap()

Create an inverted index instance

In [23]:
# Create inverted index instance
inverted = InvertedIndex()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

inverted.N = N

inverted.doc_len = dl

# write the global stats out
inverted.write_index('.', 'index')




In [24]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing


def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [25]:
## test the final index
pl = read_posting_list(inverted, 'hello')
print(pl)
# assert pl[-3] == (2204, 2)
# assert len(pl) == 18

[(316, 1), (856, 3), (880, 1), (1140, 1), (1242, 7), (1335, 3), (1446, 1), (1451, 4), (1453, 10), (1456, 3), (1828, 1), (1969, 1), (2100, 4), (2126, 1), (2162, 2), (2217, 1), (2398, 1)]


In [126]:
import builtins as pybuiltins



# weight = tf*idf
def calculate_tf_idf(index, term, tf, doc_id):
  # doc_len =  doc_lenght(x[0], x[1])
  tf_idf = (tf/index.doc_len[doc_id])* math.log2(index.N/index.df[term])
  return tf_idf


def cosine_similarity(query, index):
    """ Returns: {doc_id:cosine score} """

    dict_cosine_sim = {}
    doc_weights_dict = {}
    query_dict = dict(term_frequency(query ,0))

    # create lists of keys and values from the query_dict
    query_list_keys = list(query_dict.keys())
    query_list_values = list(query_dict.values())

    # create list of items from index.df dictionery keys
    index_df_list_keys = list(index.df.keys())


    for term in query_list_keys:
      w_term_query = (query_dict[term][1]/len(query))* math.log2(index.N/index.df[term])

      if term in index_df_list_keys:
        posting_lst = read_posting_list(index,term)
        for doc_id, freq in posting_lst:
          w_term_doc = calculate_tf_idf(index, term, freq, doc_id)
          if doc_id in dict_cosine_sim.keys():
            dict_cosine_sim[doc_id] += (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] += (w_term_doc) ** 2
          else:
            dict_cosine_sim[doc_id] = (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] = (w_term_doc) ** 2
# ----------

    ls = [value[1] ** 2 for value in query_list_values]
    # print(dict_cosine_sim)
    # print(doc_weights_dict)
    # total_sum = pybuiltins.sum(ls)
    # print(pybuiltins.sum(value[1] ** 2 for value in query_list_values))

    # The dictionary - "doc_weights_dict" holds for each document the sum of the weight of each word (from the query) squared. ( sum(for each word i (Wij^2)) )
    for doc_id in list(dict_cosine_sim.keys()):
      Word_doc_id__weight = doc_weights_dict[doc_id];
      dict_cosine_sim[doc_id] /= ( math.sqrt(pybuiltins.sum(value[1] ** 2 for value in query_list_values) * Word_doc_id__weight ))



    sorted_docs = sorted(dict_cosine_sim.items(), key=lambda x: x[1], reverse=True)
    # return sorted_docs

    print(sorted_docs)
    return dict_cosine_sim


Search body

In [89]:
def search(query):
  """ Returns: n best docs """
  # query_filterd = tokenize_stemming(query)
  cosSim_score = cosine_similarity(query,inverted)
  return cosSim_score


In [127]:
query = "clarity"
res = search(query)
# print(search)
# print(inverted.doc_len)
# result = search(query)
# print(term_frequency("clarity" ,0))
# print(result)


[(1525, 0.8279798976028251), (573, 0.827979897602825), (593, 0.827979897602825), (653, 0.827979897602825), (752, 0.827979897602825), (852, 0.827979897602825), (874, 0.827979897602825), (1130, 0.827979897602825), (1171, 0.827979897602825), (1500, 0.827979897602825), (1814, 0.827979897602825), (2032, 0.827979897602825), (2144, 0.827979897602825), (2161, 0.827979897602825), (2195, 0.827979897602825), (2204, 0.827979897602825), (2303, 0.827979897602825), (2414, 0.8279798976028249)]


In [75]:
## test the final index
res_dict = get_best_docs(inverted, 'hello')
print(res_dict)

NameError: name 'get_best_docs' is not defined

**search body**